In [118]:
import pandas as pd
import requests
import json
from pymongo import MongoClient
from pymongo import GEOSPHERE
import seaborn as sns
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from dotenv import load_dotenv
import os
from functools import reduce
import operator
load_dotenv()

True

In [119]:
### FOURSQUARE
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [120]:
client = MongoClient("localhost:27017")

In [121]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [122]:
db = client.get_database("ironhack")

In [123]:
db.create_collection("gaming_companies")

CollectionInvalid: collection gaming_companies already exists

In [124]:
companies = db.get_collection("companies")
gaming_proj = {"_id": 0, "name": 1, "offices.latitude": 1, "offices.longitude": 1, "number_of_employees": 1}
gaming_companies = list(companies.find({"number_of_employees": {"$gt": 50}, "total_money_raised": {"$gte": "$1M"}, "offices.zip_code": {"$exists": True}, "offices.city": {"$exists": True}}, gaming_proj))
gaming_companies

[{'name': 'Digg',
  'number_of_employees': 60,
  'offices': [{'latitude': 37.764726, 'longitude': -122.394523}]},
 {'name': 'Facebook',
  'number_of_employees': 5299,
  'offices': [{'latitude': 37.41605, 'longitude': -122.151801},
   {'latitude': 53.344104, 'longitude': -6.267494},
   {'latitude': 40.7557162, 'longitude': -73.9792469}]},
 {'name': 'eBay',
  'number_of_employees': 15000,
  'offices': [{'latitude': 37.295005, 'longitude': -121.930035}]},
 {'name': 'Cisco',
  'number_of_employees': 63000,
  'offices': [{'latitude': 37.408802, 'longitude': -121.95377}]},
 {'name': 'Yahoo!',
  'number_of_employees': 13600,
  'offices': [{'latitude': 37.418531, 'longitude': -122.025485}]},
 {'name': 'Powerset',
  'number_of_employees': 60,
  'offices': [{'latitude': 37.778613, 'longitude': -122.395289}]},
 {'name': 'AddThis',
  'number_of_employees': 120,
  'offices': [{'latitude': 38.926172, 'longitude': -77.245195},
   {'latitude': 40.724604, 'longitude': -73.996876},
   {'latitude': 34.02

In [126]:
mapa_nombre_comp =  ["name"]
mapa_latitud_comp = ["offices", 0, "longitude"]
mapa_longitud_comp = ["offices", 0, "latitude"]

def getgetFromDict(diccionario,mapa):
    mapa_nombre_comp =  ["name"]
    mapa_latitud_comp = ["offices", "longitude"]
    mapa_longitud_comp = ["offices", "latitude"]
    return reduce(operator.getitem,mapa,diccionario)

companies_list = []
for dic in gaming_companies:
    paralista = {}
    paralista["name"] = getgetFromDict(dic, mapa_nombre_comp)
    paralista["latitud"]= getgetFromDict(dic, mapa_latitud_comp)
    paralista["longitud"] = getgetFromDict(dic,mapa_longitud_comp)
    companies_list.append(paralista)
    

documents = []
for diccionario in companies_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
companies_venues = pd.DataFrame(documents)
companies_venues["place"] = "company"
companies_venues.head()

,name,location,place
0,Digg,"{'type': 'Point', 'coordinates': [-122.394523,...",company
1,Facebook,"{'type': 'Point', 'coordinates': [-122.151801,...",company
2,eBay,"{'type': 'Point', 'coordinates': [-121.930035,...",company
3,Cisco,"{'type': 'Point', 'coordinates': [-121.95377, ...",company
4,Yahoo!,"{'type': 'Point', 'coordinates': [-122.025485,...",company


In [127]:
companies_venues.to_json("companies_venues.json", orient="records")

In [128]:
#We create a new collection in Mongo, with all the companies based in San Francisco. 
gaming_companies = db.get_collection("gaming_companies")
gaming_companies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'gaming_companies')

In [129]:
sf_companies = list(gaming_companies.find())
sf_companies
san_francisco = pd.DataFrame(sf_companies)
latitude = [lat["offices"][0]["latitude"] for lat in sf_companies]
longitude = [long["offices"][0]["longitude"] for long in sf_companies]
zip_code = [code["offices"][0]["zip_code"] for code in sf_companies]
address = [point["offices"][0]["address1"] for point in sf_companies]

#De aquí añadimos las listas que nos interesen al DataFrame de la ciudad de San Francisco.
san_francisco["latitude"] = latitude
san_francisco["longitude"] = longitude
san_francisco["zip_code"] = zip_code
san_francisco["address"] = address

san_francisco = san_francisco[san_francisco["latitude"].notna()]
san_francisco = san_francisco[san_francisco["longitude"].notna()]

san_francisco.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,milestones,video_embeds,screenshots,external_links,partners,ipo,latitude,longitude,zip_code,address
0,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,"[{'id': 9588, 'description': 'Another Digg Exe...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[117, 150], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[],NaN,37.764726,-122.394523,94107,135 Mississippi St
1,52cdef7c4bab8bd675297d94,Twitter,twitter,http://www.crunchbase.com/company/twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300.0,...,"[{'id': 1, 'description': 'Survives iPhone 3G ...","[{'embed_code': '<iframe width=""430"" height=""2...","[{'available_sizes': [[[150, 74], 'assets/imag...",[{'external_url': 'http://sec.gov/cgi-bin/brow...,[],"{'valuation_amount': 18100000000, 'valuation_c...",37.776805,-122.416924,94103,1355 Market St.
2,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,"[{'id': 3103, 'description': 'Spin-out from Eb...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 72], 'assets/imag...",[{'external_url': 'http://www.facebook.com/Stu...,[],NaN,37.775196,-122.419204,,
3,52cdef7c4bab8bd675297d97,Scribd,scribd,http://www.crunchbase.com/company/scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50.0,...,"[{'id': 4960, 'description': 'Scribd launches ...","[{'embed_code': '<script src=""http://i.cdn.tur...","[{'available_sizes': [[[150, 72], 'assets/imag...",[{'external_url': 'http://www.online-tech-tips...,[],NaN,37.789634,-122.404052,94107,539 Bryant Street
4,52cdef7c4bab8bd675297da4,Powerset,powerset,http://www.crunchbase.com/company/powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60.0,...,[],"[{'embed_code': '<object width=""400"" height=""3...",[],[{'external_url': 'http://en.wikipedia.org/wik...,[],NaN,37.778613,-122.395289,94107,475 Brannan St


In [9]:
san_francisco.groupby("category_code").count().sort_values(by = "_id", ascending = False).head(10)


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,number_of_employees,founded_year,...,milestones,video_embeds,screenshots,external_links,partners,ipo,latitude,longitude,zip_code,address
category_code,,,,,,,,,,,,,,,,,,,,,
web,189,189,189,189,189,189,188,145,105,159,...,189,189,189,189,189,2,189,189,185,184
software,94,94,94,94,94,94,94,80,53,82,...,94,94,94,94,94,3,94,94,92,92
advertising,70,70,70,70,70,70,69,56,40,61,...,70,70,70,70,70,2,70,70,70,70
games_video,51,51,51,51,51,51,50,43,34,48,...,51,51,51,51,51,2,51,51,50,50
mobile,48,48,48,48,48,48,48,36,28,41,...,48,48,48,48,48,1,48,48,47,47
enterprise,40,40,40,40,40,40,40,37,28,39,...,40,40,40,40,40,0,40,40,40,40
ecommerce,23,23,23,23,23,23,23,19,18,19,...,23,23,23,23,23,1,23,23,23,23
consulting,21,21,21,21,21,21,21,14,18,19,...,21,21,21,21,21,0,21,21,21,21
network_hosting,21,21,21,21,21,21,21,11,8,17,...,21,21,21,21,21,0,21,21,20,20


In [10]:
# Coordenadas de San Francisco
initial_lat =  37.773972
initial_long = -122.431297

In [11]:
sf_map = folium.Map(location = [initial_lat, initial_long], zoom_start = 15)
for i, row in san_francisco.iterrows():
    distrito = {
        "location":[row["latitude"], row["longitude"]],
        "tooltip": [row["name"], row["zip_code"]]}
    if row["name"] == "Shattered Reality Interactive":
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "map-pin",
                    icon_color = "black")
    elif row["category_code"] == "games_video":
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "gamepad",
                    icon_color = "black")
    else: 
        icon = Icon(color = "green",
        prefix = "fas",
        icon = "map-marker-alt",
        icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(sf_map)
sf_map                

In [13]:
our_firm = san_francisco[san_francisco["name"] == "Shattered Reality Interactive"]
our_firm.drop(index = 533)
our_lat = our_firm.latitude
our_lon = our_firm.longitude

In [14]:
print(our_lat, our_lon)


484    37.783046
533    37.783046
Name: latitude, dtype: float64 484   -122.394067
533   -122.394067
Name: longitude, dtype: float64


In [15]:
#Geoquery inicial

sf = {'type': 'Point', 'coordinates': [37.783046, -122.394067]}
url_query = 'https://api.foursquare.com/v2/venues/explore'
parametros = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{sf.get('coordinates')[0]},{sf.get('coordinates')[1]}",
    "query": "basketball", 
    "limit": 100    
}

In [16]:
resp = requests.get(url= url_query, params = parametros).json()


In [17]:
data = resp.get("response").get("groups")[0].get("items")

In [18]:
#Vamos a sacar el primer DF
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue", "location", "lng"]
mapa_longitud = ["venue", "location", "lat"]

def getFromDict(diccionario,mapa):
    
    return reduce(operator.getitem,mapa,diccionario)

basket_list = []
for dic in data:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    basket_list.append(paralista)
    

documents = []
for diccionario in basket_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
basket_venues = pd.DataFrame(documents)
basket_venues["place"] = "basketball court"
basket_venues.head()

,name,location,place
0,Mission Creek Park,"{'type': 'Point', 'coordinates': [-122.3939652...",basketball court
1,Berry Basketball Courts,"{'type': 'Point', 'coordinates': [-122.3979310...",basketball court
2,Equinox Sports Club San Francisco,"{'type': 'Point', 'coordinates': [-122.4045675...",basketball court
3,YMCA,"{'type': 'Point', 'coordinates': [-122.3920834...",basketball court
4,Chase Center,"{'type': 'Point', 'coordinates': [-122.387463,...",basketball court


In [19]:
basket_venues.to_json("basket_venues_sf.json", orient="records")

In [20]:
#Hasta aquí, la manera para sacar los jsons.

In [23]:
latitude = [lat["latitud"] for lat in basket_list]
longitude = [long["longitud"] for long in basket_list]

In [24]:
basket_venues["latitude"] = longitude
basket_venues["longitude"] = latitude
basket_venues

,name,location,place,latitude,longitude
0,Mission Creek Park,"{'type': 'Point', 'coordinates': [-122.3939652...",basketball court,37.772888,-122.393965
1,Berry Basketball Courts,"{'type': 'Point', 'coordinates': [-122.3979310...",basketball court,37.771911,-122.397931
2,Equinox Sports Club San Francisco,"{'type': 'Point', 'coordinates': [-122.4045675...",basketball court,37.786649,-122.404568
3,YMCA,"{'type': 'Point', 'coordinates': [-122.3920834...",basketball court,37.792651,-122.392083
4,Chase Center,"{'type': 'Point', 'coordinates': [-122.387463,...",basketball court,37.767864,-122.387463
5,Victoria Manalo Draves Park,"{'type': 'Point', 'coordinates': [-122.4062288...",basketball court,37.777313,-122.406229
6,Basketball Court at Avalon II,"{'type': 'Point', 'coordinates': [-122.3943967...",basketball court,37.775857,-122.394397
7,Pacific Rim Professional Basketball Association,"{'type': 'Point', 'coordinates': [-122.3933868...",basketball court,37.791031,-122.393387
8,Soma Rec Center,"{'type': 'Point', 'coordinates': [-122.4063520...",basketball court,37.779012,-122.406352
9,Mission Bay Courts,"{'type': 'Point', 'coordinates': [-122.3977000...",basketball court,37.771862,-122.397700


In [154]:
#Añadimos basketball venues al map de Folium
for i, row in basket_venues.iterrows():
    basket = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip": [row["name"]]}
    if row["name"] == "Chase Center":
        icon = Icon(color = "white",
        prefix = "fas",
        icon = "basketball-ball",
        icon_color = "black")
    else:
        icon = Icon(color = "orange",
        prefix = "fas",
        icon = "basketball",
        icon_color = "black")
   
    Marker(**basket,icon = icon ).add_to(sf_map)              

Basketball done, next step, try to find vegan restaurants.

In [28]:
#Sacamos vegan_venues.

parametros_1 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{sf.get('coordinates')[0]},{sf.get('coordinates')[1]}",
    "query": "vegan", 
    "limit": 100    
}

In [29]:
resp_1 = requests.get(url= url_query, params = parametros_1).json()
data_1 = resp_1.get("response").get("groups")[0].get("items")
data_1

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '57fbb40ecd1057f331ffd3e3',
   'name': 'sweetgreen',
   'contact': {},
   'location': {'address': '171 2nd St',
    'lat': 37.7872625,
    'lng': -122.3986346,
    'labeledLatLngs': [{'label': 'display',
      'lat': 37.7872625,
      'lng': -122.3986346}],
    'distance': 617,
    'postalCode': '94105',
    'cc': 'US',
    'city': 'San Francisco',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['171 2nd St',
     'San Francisco, CA 94105',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1d3941735',
     'name': 'Vegetarian / Vegan Restaurant',
     'pluralName': 'Vegetarian / Vegan Restaurants',
     'shortName': 'Vegetarian / Vegan',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
      'suffix': '.png'},
     'primary': True}],
   'verified'

In [31]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

vegan_list = []
for dic in data_1:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    vegan_list.append(paralista)
    

documents = []
for diccionario in vegan_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
vegan_venues = pd.DataFrame(documents)
vegan_venues["place"] = "vegan"
vegan_venues.head()

,name,location,place
0,sweetgreen,"{'type': 'Point', 'coordinates': [-122.3986346...",vegan
1,South Park Cafe,"{'type': 'Point', 'coordinates': [-122.3941753...",vegan
2,Whole Foods Market,"{'type': 'Point', 'coordinates': [-122.4000804...",vegan
3,Darwin Cafe,"{'type': 'Point', 'coordinates': [-122.3961147...",vegan
4,Mixt,"{'type': 'Point', 'coordinates': [-122.399149,...",vegan


In [33]:
vegan_venues.to_json("vegan_venues_sf.json", orient="records")

In [34]:
#Sacamos starbucks

In [35]:
parametros_2 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{sf.get('coordinates')[0]},{sf.get('coordinates')[1]}",
    "query": "starbucks", 
    "limit": 100    
}

In [36]:
resp_2 = requests.get(url= url_query, params = parametros_2).json()
data_2 = resp_2.get("response").get("groups")[0].get("items")
data_2

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5ce5da80e65d0c002c9ce011',
   'name': 'Starbucks',
   'contact': {},
   'location': {'address': '299 Fremont St',
    'crossStreet': 'Folsom St',
    'lat': 37.788246,
    'lng': -122.39364,
    'labeledLatLngs': [{'label': 'display',
      'lat': 37.788246,
      'lng': -122.39364}],
    'distance': 580,
    'postalCode': '94105',
    'cc': 'US',
    'neighborhood': 'SoMa',
    'city': 'San Francisco',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['299 Fremont St (Folsom St)',
     'San Francisco, CA 94105',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1e0931735',
     'name': 'Coffee Shop',
     'pluralName': 'Coffee Shops',
     'shortName': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'},
     'p

In [37]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

starbucks_list = []
for dic in data_2:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    starbucks_list.append(paralista)
    

documents = []
for diccionario in starbucks_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
starbucks_venues = pd.DataFrame(documents)
starbucks_venues["place"] = "starbucks"
starbucks_venues.head()

,name,location,place
0,Starbucks,"{'type': 'Point', 'coordinates': [-122.39364, ...",starbucks
1,Starbucks,"{'type': 'Point', 'coordinates': [-122.3966218...",starbucks
2,Starbucks,"{'type': 'Point', 'coordinates': [-122.394111,...",starbucks
3,Starbucks,"{'type': 'Point', 'coordinates': [-122.3966867...",starbucks
4,Starbucks,"{'type': 'Point', 'coordinates': [-122.39561, ...",starbucks


In [38]:
starbucks_venues.to_json("starbucks_venues_sf.json", orient="records")

In [39]:
#Sacamos las guarderías

In [40]:
parametros_3 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{sf.get('coordinates')[0]},{sf.get('coordinates')[1]}",
    "query": "school", 
    "limit": 100    
}

In [41]:
resp_3 = requests.get(url= url_query, params = parametros_3).json()
data_3 = resp_3.get("response").get("groups")[0].get("items")
data_3

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5bccd1a198fbfc002c48f9dd',
   'name': 'The Grotto',
   'contact': {},
   'location': {'address': '490 2nd St',
    'lat': 37.78306,
    'lng': -122.39408,
    'labeledLatLngs': [{'label': 'display',
      'lat': 37.78306,
      'lng': -122.39408}],
    'distance': 1,
    'postalCode': '94107',
    'cc': 'US',
    'city': 'San Francisco',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['490 2nd St',
     'San Francisco, CA 94107',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d13b941735',
     'name': 'School',
     'pluralName': 'Schools',
     'shortName': 'School',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   'stats': {'tipCount': 0,
    'usersCount': 0,
    'check

In [42]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

school_list = []
for dic in data_3:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    school_list.append(paralista)
    

documents = []
for diccionario in school_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
school_venues = pd.DataFrame(documents)
school_venues["place"] = "school"
school_venues.head()

,name,location,place
0,The Grotto,"{'type': 'Point', 'coordinates': [-122.39408, ...",school
1,San Francisco Institute of Esthetics and Cosme...,"{'type': 'Point', 'coordinates': [-122.40703, ...",school
2,WestEd,"{'type': 'Point', 'coordinates': [-122.3984850...",school
3,INSEAD - San Francisco Hub,"{'type': 'Point', 'coordinates': [-122.3938674...",school
4,Bluemix Garage,"{'type': 'Point', 'coordinates': [-122.3966293...",school


In [43]:
school_venues.to_json("school_venues_sf.json", orient="records")

In [ ]:
#Sacamos aeropuertos cercanos

In [45]:
parametros_4 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{sf.get('coordinates')[0]},{sf.get('coordinates')[1]}",
    "query": "airport", 
    "limit": 100    
}

In [46]:
resp_4 = requests.get(url= url_query, params = parametros_4).json()
data_4 = resp_4.get("response").get("groups")[0].get("items")
data_4

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '41059b00f964a520850b1fe3',
   'name': 'San Francisco International Airport (SFO) (San Francisco International Airport)',
   'contact': {},
   'location': {'address': 'N McDonnell Rd',
    'crossStreet': 'at Link Rd',
    'lat': 37.6167130000997,
    'lng': -122.38709449768066,
    'labeledLatLngs': [{'label': 'display',
      'lat': 37.6167130000997,
      'lng': -122.38709449768066}],
    'distance': 18526,
    'postalCode': '94128',
    'cc': 'US',
    'city': 'San Francisco',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['N McDonnell Rd (at Link Rd)',
     'San Francisco, CA 94128',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1ed931735',
     'name': 'Airport',
     'pluralName': 'Airports',
     'shortName': 'Airport',
     'icon': {'prefix': 'https://ss3.4sqi.net/img

In [47]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

airport_list = []
for dic in data_4:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    airport_list.append(paralista)
    

documents = []
for diccionario in airport_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
airport_venues = pd.DataFrame(documents)
airport_venues["place"] = "airport"
airport_venues.head()

,name,location,place
0,San Francisco International Airport (SFO) (San...,"{'type': 'Point', 'coordinates': [-122.3870944...",airport
1,GOGO JETS - San Francisco Private Jet Charter,"{'type': 'Point', 'coordinates': [-122.3979440...",airport
2,San Francisco Passport Agency,"{'type': 'Point', 'coordinates': [-122.3978348...",airport
3,American Airporter,"{'type': 'Point', 'coordinates': [-122.40485, ...",airport
4,Top Airport Parking,"{'type': 'Point', 'coordinates': [-122.3965818...",airport


In [48]:
airport_venues.to_json("airport_venues_sf.json", orient="records")

In [41]:
parametros_5 = {"client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{sf.get('coordinates')[0]},{sf.get('coordinates')[1]}",
    "query": "pet hairdresser", 
    "limit": 100    
}

In [42]:
resp_5 = requests.get(url= url_query, params = parametros_5).json()
data_5 = resp_5.get("response").get("groups")[0].get("items")
data_5

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '51ad27a3498eb25779f74f0a',
   'name': 'P.L.A.Y. PET LIFESTYLE AND YOU INC',
   'contact': {},
   'location': {'address': '246 2nd St Ste A',
    'crossStreet': '2nd At Folsom',
    'lat': 37.7859399,
    'lng': -122.3974409,
    'labeledLatLngs': [{'label': 'display',
      'lat': 37.7859399,
      'lng': -122.3974409}],
    'distance': 438,
    'postalCode': '94105',
    'cc': 'US',
    'city': 'San Francisco',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['246 2nd St Ste A (2nd At Folsom)',
     'San Francisco, CA 94105',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d100951735',
     'name': 'Pet Store',
     'pluralName': 'Pet Stores',
     'shortName': 'Pet Store',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
      'suffix': '.png'}

In [43]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

dog_list = []
for dic in data_5:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    dog_list.append(paralista)
    

documents = []
for diccionario in dog_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
dog_venues = pd.DataFrame(documents)
dog_venues["place"] = "dog"
dog_venues.head()

,name,location,place
0,P.L.A.Y. PET LIFESTYLE AND YOU INC,"{'type': 'Point', 'coordinates': [-122.3974409...",dog
1,Suite 54 - Big Heart Pet Brands,"{'type': 'Point', 'coordinates': [-122.3896115...",dog
2,Woodlands Pet Shop,"{'type': 'Point', 'coordinates': [-122.3923, 3...",dog
3,Jeffrey's Natural Pet Foods,"{'type': 'Point', 'coordinates': [-122.4112558...",dog
4,Maddie's Pet Adoption Center,"{'type': 'Point', 'coordinates': [-122.4122104...",dog


In [62]:
dog_venues.to_json("dog_venues_sf.json", orient="records")

In [ ]:
#Buscar discos o clubs

In [130]:
parametros_6 = {"client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{sf.get('coordinates')[0]},{sf.get('coordinates')[1]}",
    "query": "club", 
    "limit": 100    
}

In [131]:
resp_6 = requests.get(url= url_query, params = parametros_6).json()
data_6 = resp_6.get("response").get("groups")[0].get("items")
data_6

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e6ad115e4cdb37551c1335d',
   'name': 'San Francisco Giants Home Clubhouse',
   'contact': {},
   'location': {'address': '24 Willie Mays Plz',
    'lat': 37.77825684452602,
    'lng': -122.39117295999083,
    'labeledLatLngs': [{'label': 'display',
      'lat': 37.77825684452602,
      'lng': -122.39117295999083}],
    'distance': 590,
    'postalCode': '94107',
    'cc': 'US',
    'city': 'San Francisco',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['24 Willie Mays Plz',
     'San Francisco, CA 94107',
     'United States']},
   'categories': [{'id': '52e81612bcbc57f1066b7a35',
     'name': 'Club House',
     'pluralName': 'Club Houses',
     'shortName': 'Club',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
      'suffix': '.png'},
     'primary': Tru

In [132]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

club_list = []
for dic in data_6:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    club_list.append(paralista)
    

documents = []
for diccionario in club_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
club_venues = pd.DataFrame(documents)
club_venues["place"] = "club"
club_venues.head()

,name,location,place
0,San Francisco Giants Home Clubhouse,"{'type': 'Point', 'coordinates': [-122.3911729...",club
1,Millennium Tower Club Level,"{'type': 'Point', 'coordinates': [-122.3960379...",club
2,Gold Club,"{'type': 'Point', 'coordinates': [-122.3996934...",club
3,Crunch - Yerba Buena,"{'type': 'Point', 'coordinates': [-122.3987623...",club
4,The Grove,"{'type': 'Point', 'coordinates': [-122.40196, ...",club


In [133]:
club_venues.to_json("club_venues_sf.json", orient="records")

In [78]:
sf_venues = db.get_collection("sf_venues")

In [109]:
#Desde aquí sacamos las coordenadas del lugar que tenemos pensado coger y ampliamos a un radio de 10 km para buscar las basketball venues. 

sf_coordinates = [-122.394067, 37.783046]
coord_point = {"type":"Point", "coordinates": sf_coordinates}

In [80]:
query = {"location": {"$near": {"$geometry": coord_point,"$minDistance": 0  , "$maxDistance": 500}}}

In [81]:
query_final = list(sf_venues.find(query))
query_final

[{'_id': ObjectId('607da03a9fa8440ab6c539b8'),
  'name': 'Flurry',
  'location': {'type': 'Point', 'coordinates': [-122.394067, 37.783046]},
  'place': 'company'},
 {'_id': ObjectId('607d3edc74ad270aa404c4a4'),
  'name': 'The Grotto',
  'location': {'type': 'Point', 'coordinates': [-122.39408, 37.78306]},
  'place': 'school'},
 {'_id': ObjectId('607d3ccb74ad270aa404c439'),
  'name': '21st Amendment Brewery & Restaurant',
  'location': {'type': 'Point',
   'coordinates': [-122.3925877625, 37.7823464051]},
  'place': 'vegan'},
 {'_id': ObjectId('607d888974ad270aa404c565'),
  'name': 'South Park Commons',
  'location': {'type': 'Point',
   'coordinates': [-122.3944623181, 37.7815946128]},
  'place': 'club'},
 {'_id': ObjectId('607d3ccb74ad270aa404c42b'),
  'name': 'South Park Cafe',
  'location': {'type': 'Point',
   'coordinates': [-122.3941753066, 37.7815500691]},
  'place': 'vegan'},
 {'_id': ObjectId('607d3edc74ad270aa404c4b1'),
  'name': 'Bright Horizons at 2nd Street',
  'location':

In [106]:
sf_final = pd.DataFrame(query_final)
len(sf_final)

35

In [113]:
latitude = [lat["location"]["coordinates"][1] for lat in query_final]
longitude = [long["location"]["coordinates"][0] for long in query_final]

#De aquí añadimos las listas que nos interesen al DataFrame de la ciudad de San Francisco.
sf_final["latitude"] = latitude
sf_final["longitude"] = longitude


sf_final = sf_final[sf_final["latitude"].notna()]
sf_final = sf_final[sf_final["longitude"].notna()]

sf_final_companies = sf_final[sf_final["place"] == "company"]
sf_final_companies

,_id,name,location,place,latitude,longitude
0,607da03a9fa8440ab6c539b8,Flurry,"{'type': 'Point', 'coordinates': [-122.394067,...",company,37.783046,-122.394067
6,607da03a9fa8440ab6c53947,Slide,"{'type': 'Point', 'coordinates': [-122.392144,...",company,37.781427,-122.392144
7,607da03a9fa8440ab6c53991,Splunk,"{'type': 'Point', 'coordinates': [-122.390945,...",company,37.782740,-122.390945
9,607da03a9fa8440ab6c53b2e,ZoomSystems,"{'type': 'Point', 'coordinates': [-122.391061,...",company,37.781689,-122.391061
19,607da03a9fa8440ab6c53953,PowerReviews,"{'type': 'Point', 'coordinates': [-122.393041,...",company,37.779558,-122.393041
23,607da03a9fa8440ab6c53abe,Skout,"{'type': 'Point', 'coordinates': [-122.3912492...",company,37.780054,-122.391249
28,607da03a9fa8440ab6c53a77,Smule,"{'type': 'Point', 'coordinates': [-122.391236,...",company,37.779624,-122.391236
29,607da03a9fa8440ab6c53973,imeem,"{'type': 'Point', 'coordinates': [-122.391236,...",company,37.779624,-122.391236
32,607da03a9fa8440ab6c53a13,Cloudmark,"{'type': 'Point', 'coordinates': [-122.39071, ...",company,37.779507,-122.390710
33,607da03a9fa8440ab6c53961,TokBox,"{'type': 'Point', 'coordinates': [-122.39071, ...",company,37.779507,-122.390710


In [115]:
sf_map = folium.Map(location = [37.783046, -122.394067], zoom_start = 15)
for i, row in sf_final_companies.iterrows():
    distrito = {
        "location":[row["latitude"], row["longitude"]],
        "tooltip": [row["name"], "aquí tienes tus putos sitios, Pablo"]}
    
    icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "map-pin",
                    icon_color = "black")
        
        
    
    Marker(**distrito,icon = icon ).add_to(sf_map)
sf_map      